# Multilayer perceptron

Here’s an example of my script in which I make multilayer perceptron on some financial data. The data has 53 independent variables (X) and one independent variable (y). It is time series data and X exhibits plenty of time-series related information. The goal is to fit MP as good as possible.

We import the packages:

In [1]:
import numpy as np
import sys
import theano
import theano.tensor as T
import matplotlib.pyplot as plt ### Not needed at the moment, but can be useful to display plots.
import lasagne
from lasagne.nonlinearities import leaky_rectify, softmax, sigmoid, tanh


Read data in and print the amount of data we are dealing with:

In [2]:

input_data_all = np.load('train_sample.npy')
output_data_all= np.load('train_target.npy')
n_samples= input_data_all.shape[0]
print(n_samples)

96674


Shuffle two arrays together while preserving their correspondence - so we can mix target and dependent variable together.


In [3]:
def shuffle_together(input_1, input_2):
    if input_1.shape[0]!= input_2.shape[0]:
        print ("Problem, y and x array are not of the same shape.")
        return None
    c= np.arange(input_1.shape[0])
    np.random.shuffle(c)
    return input_1[c], input_2[c], c

Create Lasagne layers given array of different numbers of dimensions, input to layers, x_variable and which nonlinearities to choose.

In [4]:
def create_layers(l_dim,my_input,x_variable,l_nonlinearities):
    nr_layers = len(l_dim)
    layers = [] ### Create an array where we will input layers, starting with input layer.
    layers.append(lasagne.layers.InputLayer(shape=my_input.shape, input_var=x_variable))
    if len(l_dim)==1:
        return(layers)
    else:
        i = 1
        while i<len(l_dim):
            layers.append(lasagne.layers.DenseLayer(layers[i-1], num_units=l_dim[i],nonlinearity=l_nonlinearities[i]))
            i += 1
        return(layers)
        

Shuffle input and output data and create Theano variables for input and target minibatch;

In [5]:
input_data_all, output_data_all, _ = shuffle_together(input_data_all, output_data_all)
n_validation_samples= int(0.1*n_samples)
validation_input= input_data_all[-1*n_validation_samples:]
validation_output= output_data_all[-1*n_validation_samples:]
n_train_samples= n_samples-n_validation_samples
train_input= input_data_all[:n_train_samples]
train_output= output_data_all[:n_train_samples]
print(train_input.shape)
print(train_output.shape)
x=T.matrix('x', dtype= theano.config.floatX)
y=T.vector('y', dtype= theano.config.floatX)


(87007, 53)
(87007,)


Create layers. Here we set up how layers will be assembled. Also, we get the last layer output.

In [8]:
layers_dimensions = [input_data_all.shape[1],40,25,15,8,4,1]
### Above means that first layer will have 40, second 25, etc. dimensions.
layers_nonlinearities = [None,lasagne.nonlinearities.rectify,lasagne.nonlinearities.tanh,
                     lasagne.nonlinearities.tanh,lasagne.nonlinearities.rectify,lasagne.nonlinearities.linear,None]
network = create_layers(l_dim = layers_dimensions,my_input=train_input,x_variable=x,l_nonlinearities=layers_nonlinearities)
output_layer = network[-1]
output_val =  lasagne.layers.get_output(output_layer)

Create loss function. After that, define updates and how they will be made.

In [11]:
prediction = lasagne.layers.get_output(output_layer)
loss = lasagne.objectives.squared_error(prediction, y)
loss = lasagne.objectives.aggregate(loss, mode = 'mean')
params = lasagne.layers.get_all_params(output_layer, trainable=True)
updates = lasagne.updates.nesterov_momentum(loss, params, learning_rate=0.0000001, momentum=0.9)

We define sample function (calculates output) and train_fn, which updates the parameters.

In [13]:
sample_function= theano.function(inputs= [x], outputs=[output_val])
train_fn = theano.function([x, y], loss, updates=updates)


WARNING (theano.gof.compilelock): Overriding existing lock by dead process '15748' (I am process '11284')


Example of usage of sample function:

In [15]:
[sample_out]= sample_function(validation_input)
sample_out

array([[-0.03904202],
       [ 0.07121486],
       [ 0.02971448],
       ...,
       [ 0.03272565],
       [ 0.06261622],
       [ 0.03850413]])

Quick evaluation of validation output without trained parameters (so, default params):

In [16]:
print(np.std(sample_out))
print(np.mean(sample_out))

0.1149251857400123
0.02464972597834169


Below is the main code. We choose batch size, num epochs, save the results in r2_error object. We shuffle training data before each epoch. In each epoch we then go through shuffled data, train model on each input/output batch. In the end, we make calculation every x epochs on how well the validation set performs and print out the performance. In this way, we follow progress on validation output.

In [17]:
batch_size= 200
num_epochs = 50000
n_batches= train_input.shape[0]//batch_size
total_it= num_epochs*n_batches
if 'r2_error' in locals():
    r2_error= np.concatenate([r2_error, np.zeros(total_it)], axis=0)
else:
    r2_error= np.zeros(total_it)
previous_count=0
train_input, train_output, _= shuffle_together(train_input, train_output)
# train network (assuming you've got some training data in numpy arrays)
ss_tot = np.sum((validation_output-np.mean(validation_output))**2)
for epoch in range(num_epochs):
    train_loss = 0
    for it in range(n_batches):
        curr_it= n_batches*epoch+it +previous_count
        input_batch= train_input[it*batch_size:(it+1)*batch_size,:]
        output_batch= train_output[it*batch_size:(it+1)*batch_size]  
        train_loss += train_fn(input_batch, output_batch)
        
    [sample_out]= sample_function(validation_input)
    SS_res = np.sum((sample_out-validation_output)**2)
    r2_error[curr_it]= 1 - SS_res/ss_tot
    print("Epoch %d: Loss %g" % (epoch + 1, train_loss / len(input_data_all)))
    print("iteration:",curr_it,"r2_error:",r2_error[curr_it])
    train_input, train_output, _= shuffle_together(train_input, train_output)

Epoch 1: Loss 0.0113295
iteration: 434 r2_error: -574786.536903361
Epoch 2: Loss 0.0110627
iteration: 869 r2_error: -562417.8391131927
Epoch 3: Loss 0.0108323
iteration: 1304 r2_error: -551496.2178449756
Epoch 4: Loss 0.0106286
iteration: 1739 r2_error: -541618.8851601696
Epoch 5: Loss 0.010443
iteration: 2174 r2_error: -532529.351458485
Epoch 6: Loss 0.0102688
iteration: 2609 r2_error: -523914.6664194066
Epoch 7: Loss 0.0101021
iteration: 3044 r2_error: -515568.29449457163
Epoch 8: Loss 0.00993871
iteration: 3479 r2_error: -507299.2015407688
Epoch 9: Loss 0.00977594
iteration: 3914 r2_error: -499024.0634955908
Epoch 10: Loss 0.00961317
iteration: 4349 r2_error: -490679.59464380343
Epoch 11: Loss 0.00945015
iteration: 4784 r2_error: -482254.5404002292
Epoch 12: Loss 0.00928663
iteration: 5219 r2_error: -473860.0078922612
Epoch 13: Loss 0.00912343
iteration: 5654 r2_error: -465362.940379234
Epoch 14: Loss 0.008958
iteration: 6089 r2_error: -456633.608811546
Epoch 15: Loss 0.00878911
ite

Epoch 115: Loss 0.000370088
iteration: 50024 r2_error: -19396.45194320973
Epoch 116: Loss 0.000367988
iteration: 50459 r2_error: -19285.789221362065
Epoch 117: Loss 0.000365926
iteration: 50894 r2_error: -19183.26004813835
Epoch 118: Loss 0.000364032
iteration: 51329 r2_error: -19087.719545854732
Epoch 119: Loss 0.000362259
iteration: 51764 r2_error: -18998.4866894738
Epoch 120: Loss 0.00036059
iteration: 52199 r2_error: -18914.612197104598
Epoch 121: Loss 0.000358981
iteration: 52634 r2_error: -18835.33600186171
Epoch 122: Loss 0.000357524
iteration: 53069 r2_error: -18760.24977303885
Epoch 123: Loss 0.000356063
iteration: 53504 r2_error: -18688.88320911371
Epoch 124: Loss 0.000354748
iteration: 53939 r2_error: -18620.876030568998
Epoch 125: Loss 0.000353446
iteration: 54374 r2_error: -18555.911268251668
Epoch 126: Loss 0.000352184
iteration: 54809 r2_error: -18493.78564655752
Epoch 127: Loss 0.000351012
iteration: 55244 r2_error: -18434.05673882654
Epoch 128: Loss 0.00034989
iteratio

Epoch 225: Loss 0.000301214
iteration: 97874 r2_error: -15918.33794227748
Epoch 226: Loss 0.000300952
iteration: 98309 r2_error: -15903.625345552842
Epoch 227: Loss 0.000300673
iteration: 98744 r2_error: -15889.003399936522
Epoch 228: Loss 0.000300399
iteration: 99179 r2_error: -15874.487501187015
Epoch 229: Loss 0.000300101
iteration: 99614 r2_error: -15860.006006342739
Epoch 230: Loss 0.000299854
iteration: 100049 r2_error: -15845.69629823974
Epoch 231: Loss 0.00029956
iteration: 100484 r2_error: -15831.47534795257
Epoch 232: Loss 0.000299283
iteration: 100919 r2_error: -15817.387025900676
Epoch 233: Loss 0.000298962
iteration: 101354 r2_error: -15803.392051325496
Epoch 234: Loss 0.000298764
iteration: 101789 r2_error: -15789.427477359035
Epoch 235: Loss 0.000298519
iteration: 102224 r2_error: -15775.58537445446
Epoch 236: Loss 0.000298252
iteration: 102659 r2_error: -15761.815663014399
Epoch 237: Loss 0.000298001
iteration: 103094 r2_error: -15748.168640950184
Epoch 238: Loss 0.0002

Epoch 334: Loss 0.000278833
iteration: 145289 r2_error: -14724.007977515974
Epoch 335: Loss 0.000278664
iteration: 145724 r2_error: -14715.796738925214
Epoch 336: Loss 0.000278487
iteration: 146159 r2_error: -14707.599059186234
Epoch 337: Loss 0.000278367
iteration: 146594 r2_error: -14699.463577312128
Epoch 338: Loss 0.000278208
iteration: 147029 r2_error: -14691.34143844681
Epoch 339: Loss 0.000278069
iteration: 147464 r2_error: -14683.26424740856
Epoch 340: Loss 0.000277905
iteration: 147899 r2_error: -14675.230786070251
Epoch 341: Loss 0.000277757
iteration: 148334 r2_error: -14667.20629943145
Epoch 342: Loss 0.000277632
iteration: 148769 r2_error: -14659.21700776692
Epoch 343: Loss 0.000277466
iteration: 149204 r2_error: -14651.263038039966
Epoch 344: Loss 0.00027727
iteration: 149639 r2_error: -14643.326500255509
Epoch 345: Loss 0.000277187
iteration: 150074 r2_error: -14635.487247236384
Epoch 346: Loss 0.000277023
iteration: 150509 r2_error: -14627.630789960853
Epoch 347: Loss 0

Epoch 443: Loss 0.000265206
iteration: 192704 r2_error: -13996.76652070932
Epoch 444: Loss 0.000265088
iteration: 193139 r2_error: -13991.328918594843
Epoch 445: Loss 0.000264994
iteration: 193574 r2_error: -13985.92007905643
Epoch 446: Loss 0.000264816
iteration: 194009 r2_error: -13980.50193323313
Epoch 447: Loss 0.000264789
iteration: 194444 r2_error: -13975.12182650846
Epoch 448: Loss 0.000264685
iteration: 194879 r2_error: -13969.754707245776
Epoch 449: Loss 0.000264597
iteration: 195314 r2_error: -13964.419406657202
Epoch 450: Loss 0.000264492
iteration: 195749 r2_error: -13959.061620478364
Epoch 451: Loss 0.000264401
iteration: 196184 r2_error: -13953.770917915273
Epoch 452: Loss 0.000264295
iteration: 196619 r2_error: -13948.454273515224
Epoch 453: Loss 0.000264174
iteration: 197054 r2_error: -13943.202884612043
Epoch 454: Loss 0.000264088
iteration: 197489 r2_error: -13937.918845479971
Epoch 455: Loss 0.000263993
iteration: 197924 r2_error: -13932.67083368205
Epoch 456: Loss 0

...

Epoch 4498: Loss 0.000191069
iteration: 1956629 r2_error: -10526.054847579242
Epoch 4499: Loss 0.000191057
iteration: 1957064 r2_error: -10525.922121259013
Epoch 4500: Loss 0.000191059
iteration: 1957499 r2_error: -10525.816273393792
Epoch 4501: Loss 0.000191061
iteration: 1957934 r2_error: -10525.711055322283
Epoch 4502: Loss 0.000191047
iteration: 1958369 r2_error: -10525.540006496707
Epoch 4503: Loss 0.000191046
iteration: 1958804 r2_error: -10525.40891582987
Epoch 4504: Loss 0.000191033
iteration: 1959239 r2_error: -10525.286273288242
Epoch 4505: Loss 0.000191031
iteration: 1959674 r2_error: -10525.162402909571
Epoch 4506: Loss 0.000191014
iteration: 1960109 r2_error: -10525.028319824623
Epoch 4507: Loss 0.000191021
iteration: 1960544 r2_error: -10524.891713528961
Epoch 4508: Loss 0.000191015
iteration: 1960979 r2_error: -10524.818018506028
Epoch 4509: Loss 0.000191004
iteration: 1961414 r2_error: -10524.65085963522
Epoch 4510: Loss 0.000190997
iteration: 1961849 r2_error: -10524.5

Epoch 4604: Loss 0.000190352
iteration: 2002739 r2_error: -10513.03850455949
Epoch 4605: Loss 0.000190354
iteration: 2003174 r2_error: -10512.941778942233
Epoch 4606: Loss 0.000190338
iteration: 2003609 r2_error: -10512.828020803016
Epoch 4607: Loss 0.00019034
iteration: 2004044 r2_error: -10512.698331787818
Epoch 4608: Loss 0.000190324
iteration: 2004479 r2_error: -10512.57929623509
Epoch 4609: Loss 0.000190308
iteration: 2004914 r2_error: -10512.47355799131
Epoch 4610: Loss 0.00019032
iteration: 2005349 r2_error: -10512.342602187742
Epoch 4611: Loss 0.000190311
iteration: 2005784 r2_error: -10512.255935465855
Epoch 4612: Loss 0.000190293
iteration: 2006219 r2_error: -10512.121575860352
Epoch 4613: Loss 0.000190289
iteration: 2006654 r2_error: -10511.986179243135
Epoch 4614: Loss 0.00019028
iteration: 2007089 r2_error: -10511.85656006014
Epoch 4615: Loss 0.00019028
iteration: 2007524 r2_error: -10511.787410913857
Epoch 4616: Loss 0.000190272
iteration: 2007959 r2_error: -10511.6823846

Epoch 4710: Loss 0.000189637
iteration: 2048849 r2_error: -10501.185920428123
Epoch 4711: Loss 0.000189624
iteration: 2049284 r2_error: -10501.097854742502
Epoch 4712: Loss 0.000189611
iteration: 2049719 r2_error: -10500.990759535805
Epoch 4713: Loss 0.000189605
iteration: 2050154 r2_error: -10500.8765000063
Epoch 4714: Loss 0.000189602
iteration: 2050589 r2_error: -10500.77350957992
Epoch 4715: Loss 0.0001896
iteration: 2051024 r2_error: -10500.658864047171
Epoch 4716: Loss 0.000189593
iteration: 2051459 r2_error: -10500.565799583363
Epoch 4717: Loss 0.000189563
iteration: 2051894 r2_error: -10500.474842744386
Epoch 4718: Loss 0.000189567
iteration: 2052329 r2_error: -10500.346832483458
Epoch 4719: Loss 0.000189562
iteration: 2052764 r2_error: -10500.2784845276
Epoch 4720: Loss 0.00018957
iteration: 2053199 r2_error: -10500.13034928851
Epoch 4721: Loss 0.000189557
iteration: 2053634 r2_error: -10500.072350138462
Epoch 4722: Loss 0.000189545
iteration: 2054069 r2_error: -10499.91415389

Epoch 4816: Loss 0.000188951
iteration: 2094959 r2_error: -10490.618732396779
Epoch 4817: Loss 0.000188961
iteration: 2095394 r2_error: -10490.511995803541
Epoch 4818: Loss 0.000188939
iteration: 2095829 r2_error: -10490.365339507014
Epoch 4819: Loss 0.000188934
iteration: 2096264 r2_error: -10490.307368958674
Epoch 4820: Loss 0.000188933
iteration: 2096699 r2_error: -10490.18741769378
Epoch 4821: Loss 0.000188939
iteration: 2097134 r2_error: -10490.120131241287
Epoch 4822: Loss 0.000188916
iteration: 2097569 r2_error: -10490.032602481771
Epoch 4823: Loss 0.000188917
iteration: 2098004 r2_error: -10489.922821238704
Epoch 4824: Loss 0.000188916
iteration: 2098439 r2_error: -10489.800426418811
Epoch 4825: Loss 0.000188906
iteration: 2098874 r2_error: -10489.710533298485
Epoch 4826: Loss 0.000188894
iteration: 2099309 r2_error: -10489.589785751368
Epoch 4827: Loss 0.000188895
iteration: 2099744 r2_error: -10489.501578938503
Epoch 4828: Loss 0.000188891
iteration: 2100179 r2_error: -10489.

Epoch 4922: Loss 0.000188355
iteration: 2141069 r2_error: -10480.63229736087
Epoch 4923: Loss 0.000188343
iteration: 2141504 r2_error: -10480.57052942863
Epoch 4924: Loss 0.000188345
iteration: 2141939 r2_error: -10480.454032880481
Epoch 4925: Loss 0.000188343
iteration: 2142374 r2_error: -10480.358729020018
Epoch 4926: Loss 0.000188333
iteration: 2142809 r2_error: -10480.253337599306
Epoch 4927: Loss 0.000188325
iteration: 2143244 r2_error: -10480.155769310606
Epoch 4928: Loss 0.000188336
iteration: 2143679 r2_error: -10480.064829550798
Epoch 4929: Loss 0.000188321
iteration: 2144114 r2_error: -10479.953892423699
Epoch 4930: Loss 0.000188315
iteration: 2144549 r2_error: -10479.8621925162
Epoch 4931: Loss 0.000188307
iteration: 2144984 r2_error: -10479.790475626995
Epoch 4932: Loss 0.000188291
iteration: 2145419 r2_error: -10479.714690147985
Epoch 4933: Loss 0.000188286
iteration: 2145854 r2_error: -10479.60820125402
Epoch 4934: Loss 0.000188284
iteration: 2146289 r2_error: -10479.4775

Example of output, mean, comparing with validation output and validation mean.

In [18]:
sample_out

array([[1.61953454],
       [1.61370057],
       [1.60749781],
       ...,
       [1.59722242],
       [1.57823222],
       [1.61016677]])

In [19]:
np.mean(sample_out)

1.6002919029124505

In [20]:
validation_output

array([2.0887, 1.5718, 1.6368, ..., 1.7315, 1.5858, 1.7527])

In [21]:
np.mean(validation_output)

1.6000274893969173

In [22]:
output_layer.get_params()[0].get_value()

array([[0.25466871],
       [0.82248584],
       [0.31789015],
       [0.58203032]])

In the end, we load test set and make predictions on it.

In [23]:
test_x = np.load('test_sample.npy')#,dtype=theano.config.floatX)
test_y= np.load('test_target.npy')#,dtype=theano.config.floatX)

We train linear and random forest model beore we save predictions of all of them on test set. Predictions can then be compared.

In [24]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(train_input, train_output)
linear_prediction = reg.predict(test_x)
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(n_estimators=250,criterion="mse")
regr.fit(train_input, train_output)
rf_prediction = regr.predict(test_x)
[sample_out]= sample_function(test_x)
import pandas as pd
export = pd.DataFrame(sample_out)
export['test_output'] = test_y
export['Random_forest'] = rf_prediction
export['Linear_prediction'] = linear_prediction
export.columns = ['sample_output', 'test_output','Random_forest','Linear_prediction']
export.to_csv("model_output2.csv")

And now we save values from layers and then save them to disk. In this way, we have the whole model on disk and we can even train it further later on or simply use it at any time that is needed.

In [28]:
values = lasagne.layers.get_all_param_values(network)
values

[array([[-0.21834705,  0.02598938, -0.16139283, ..., -0.03093223,
         -0.1047376 ,  0.19104   ],
        [-0.15906565,  0.15445803,  0.09307423, ...,  0.21699212,
          0.00544925,  0.04367405],
        [-0.18275426,  0.12965921,  0.25148818, ..., -0.13819062,
         -0.0902794 ,  0.21410064],
        ...,
        [-0.05800242,  0.08811117,  0.11002376, ...,  0.24418484,
         -0.09862999, -0.05550718],
        [-0.23508345, -0.0255055 , -0.08401327, ...,  0.00355731,
          0.22810125, -0.03328349],
        [ 0.06944318, -0.00038469, -0.09439863, ..., -0.23630851,
         -0.07337331,  0.02634185]]),
 array([-8.97137765e-05,  2.28364082e-04, -2.61037093e-05, -6.46831644e-04,
        -1.26954290e-03,  3.59740815e-04, -4.03946716e-04,  6.54692558e-04,
         2.92472761e-04, -2.82740835e-04,  1.82967749e-04, -4.91721646e-05,
         5.79294066e-04,  1.21958654e-05, -1.15813989e-03, -2.19807683e-03,
         7.93815652e-04, -6.43832880e-04, -3.12911270e-04, -3.2219017

In [41]:
import pickle as pkl
with open('model5.pkl','wb') as f:
    pkl.dump(values, f)